# View Extracted Pixel Exposure Sky background and noise  from Calexp and CalexpBackground and CCD Visits for LSSTComCamSim

- Just read pandas dataframe and show plots

- author Sylvie Dagoret-Campagne
- creation date 2024-05-11
- last update 2024-05-12
- affiliation : IJCLab
- kernel : **w_2024_16**

 
``Quantum Graphs``: https://tigress-web.princeton.edu/~lkelvin/pipelines/ 

https://tigress-web.princeton.edu/~lkelvin/pipelines/current/drp_pipe/LSSTComCamSim/nightly-validation-ops-rehearsal-3/

``CalibrateTask``: 
https://github.com/lsst/pipe_tasks/blob/main/python/lsst/pipe/tasks/calibrate.py#L392-L399


``Shemas``:
https://dm.lsst.org/sdm_schemas/browser/


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time

In [ ]:
import matplotlib.dates as mdates
#month_locator = mdates.MonthLocator(interval=1)
#day_locator = mdates.DayLocator(interval=1)
#year_month_formatter = mdates.DateFormatter("%Y-%m") 

from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")





In [ ]:
from lsst.daf.butler import Butler



# INSERT YOUR auxTel/LATISS collection and tract/band HERE
butlerRoot = "/repo/embargo"
collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240311/w_2024_10/PREOPS-4985'
collections = [collection2]
collectionn = collection2
collectionStr = collectionn.replace("/", "_")
fn_ccdVisit_photocalib = f"ccdVisittractpatch_{collectionStr}_calexpinfo.csv"
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"
instrument = 'LSSTComCamSim'
skymapName = "ops_rehersal_prep_2k_v1"
where_clause = "instrument = \'" + instrument+ "\'"
NDET = 9
suptitle=  collectionStr

suptitle = collectionStr + f" inst = {instrument} "

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4


## Read inputs

### Access CCD visit table

In [ ]:
# Initiate butler from variables set above
butler = Butler(butlerRoot, collections=collections, instrument=instrument, skymap=skymapName)
camera = butler.get("camera", instrument=instrument)
print("camera_name = {}".format(camera.getName()))
print("collections = {}".format(collections))

In [ ]:
ccd_visit_table = butler.get('ccdVisitTable', dataId={'instrument': instrument}, collections = collections )    

In [ ]:
ccd_visit_table["ccdVid"] = ccd_visit_table.index -  ccd_visit_table.index[0]
ccd_visit_table["Vid"] = ccd_visit_table["visitId"]  -   ccd_visit_table["visitId"].iloc[0]

In [ ]:
#subtract the a bug number to have normal dates
ccd_visit_table["nightObs"] = ccd_visit_table.apply(lambda x: x['visitId']//100_000, axis=1)
ccd_visit_table["nightObs"]

In [ ]:
# get time
ccd_visit_table["Time"] = pd.to_datetime(ccd_visit_table['obsStart'])

In [ ]:
# get airmass
ccd_visit_table["airmass"] = ccd_visit_table["zenithDistance"].apply(lambda x: 1/np.cos(np.pi/180.*x))

In [ ]:
# select the missing quantities (the time)
ccd_visit_table_subset = ccd_visit_table[["Time","obsStart"]]

### Read pandas dataframe containing the tract-patch decomposition of visits

In [ ]:

# need the visit file because observation date is missing
file_selected_visits = os.path.join("../Visits",fn_ccdVisit_tracts_patches)
df1 = pd.read_csv(file_selected_visits,index_col=0)

### Read the dataframe with the extracted parameters from calexp,calexpBackground

In [ ]:
df = pd.read_csv(fn_ccdVisit_photocalib,index_col=0)

In [ ]:
df

In [ ]:
df.set_index("ccdVisitId",inplace=True)

In [ ]:
df

## Add to the main dataframe df, the obstime found in ccd_visit_table

In [ ]:
df = df.join(ccd_visit_table_subset)

In [ ]:
df

## Formula to bear in mind

$$
\left\{
\begin{array}{ccc} \\
m_{AB}  & = & -2.5\cdot \log_{10}(ADU) + m(ZP) \\ \\
F(Jy) & = & F_{AB}^{ref}(Jy) \cdot 10^{-0.4\cdot m(ZP)}\cdot ADU
\end{array}
\right.
$$

## Additionnal processing

In [ ]:
df["magzero_dt"] = df["magzero"].apply(lambda x : x+2.5*np.log10(30))
df["zeroPoint_diff"] = df["zeroPoint"]-df["magzero_dt"]

## Add Sigma/N

In [ ]:
df["sigmatoskybkg"] = df["sigMad_ce"]/df["med_bkg"]
df["sigmatoskybkg_max"] = df["med_bkg"].apply(lambda x : 1/np.sqrt(x))

In [ ]:
df.columns

In [ ]:
df["photocalib_mag"] = df["photocalib_m"].apply(convert_fluxtomag)

In [ ]:
df["zeroPoint_photocalib_diff"] = df["zeroPoint"] - df["photocalib_mag"]
df["magzerodt_photocalib_diff"] = df["magzero_dt"] - df["photocalib_mag"]

## Split the dataframe in bands

In [ ]:
df_g = df[df["band"] == 'g']
df_r = df[df["band"] == 'r']
df_i = df[df["band"] == 'i']
df_z = df[df["band"] == 'z']
df_y = df[df["band"] == 'y']

## Sky Background

### Sky Background in g band

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(16,6),sharex=False)
title = f" $\sigma$ in Calexp divided by $median$ in CalexpBackground in LSSTComCamSim, band g"
df_g.plot.scatter(x="Time",y="sigmatoskybkg",ax=ax,marker="o",c="airmass",cmap="jet",rot=45,ylabel="ratio $\sigma$/$median$",title=title,grid=True)
df_g.plot.scatter(x="Time",y="sigmatoskybkg_max",ax=ax,marker=".",color="pink",rot=45,label="electron statistics",grid=True)
ax.xaxis.set_major_formatter(date_form)
#ax.set_ylim(0.,10)
ax.set_yscale('log')
ax.axhline(1.,lw=2,color="k",label="poissonnian limit")
ax.set_ylabel("sky background : $\sigma_N/N$")
ax.legend()
plt.suptitle(suptitle)
figname = "calexp_sigmaNbyN_bandG_LSSTComCamSim.png"
plt.tight_layout()
plt.savefig(figname)
plt.show()

### Sky Background in r band

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(16,6),sharex=False)
title = f" $\sigma$ in Calexp divided by $median$ in CalexpBackground in LSSTComCamSim, band r"
df_r.plot.scatter(x="Time",y="sigmatoskybkg",ax=ax,marker="o",c="airmass",cmap="jet",rot=45,ylabel="ratio $\sigma$/$median$",title=title,grid=True)
df_r.plot.scatter(x="Time",y="sigmatoskybkg_max",ax=ax,marker=".",color="pink",rot=45,label="electron statistics",grid=True)
ax.xaxis.set_major_formatter(date_form)
#ax.set_ylim(0.,10)
ax.set_yscale('log')
ax.axhline(1.,lw=2,color="k",label="poissonnian limit")
ax.legend()
plt.suptitle(suptitle)
ax.set_ylabel("sky background : $\sigma_N/N$")
figname = "calexp_sigmaNbyN_bandR_LSSTComCamSim.png"
plt.tight_layout()
plt.savefig(figname)
plt.show()

### Sky Background in i band

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(16,6),sharex=False)
title = f" $\sigma$ in Calexp divided by $median$ in CalexpBackground in LSSTComCamSim, band i"
df_i.plot.scatter(x="Time",y="sigmatoskybkg",ax=ax,marker="o",c="airmass",cmap="jet",rot=45,ylabel="ratio $\sigma$/$median$",title=title,grid=True)
df_i.plot.scatter(x="Time",y="sigmatoskybkg_max",ax=ax,marker=".",color="pink",rot=45,label="electron statistics",grid=True)
ax.xaxis.set_major_formatter(date_form)
#ax.set_ylim(0.,10)
ax.set_yscale('log')
ax.axhline(1.,lw=2,color="k",label="poissonnian limit")
ax.legend()
plt.suptitle(suptitle)
ax.set_ylabel("sky background : $\sigma_N/N$")
figname = "calexp_sigmaNbyN_bandI_LSSTComCamSim.png"
plt.tight_layout()
plt.savefig(figname)
plt.show()